# 240409

## SQL 과 ORM
> 장고 개발자는 필수적으로 sql을 알아야 한다

## 데이터 조회

### 전체 데이터 조회
- book에 저장된 모든 데이터 조회
```sql
SELECT *
FROM books;
```

```python
books = Book.objects.all()
```

### 특정 데이터 조회
- id 값에 따른 조회
```sql
SELECT *
FROM books
WHERE id = N;
```
```python
book = Book.objects.get(pk = book_pk)
```
- 특정 데이터 조회시 QUERY 확인


In [ ]:
from django.db import connection
user = User.objects.get(pk=1)
print(connection.queries)

## django의 설계 철학 (get VS filter)
- get은 모델 객체를 반환한다
  - 만약 해당하는 객체가 없으면?
    - DoesNotExist 에러 발생
      - 특정 객체가 존재한다라는 걸 보장할 수 있을때 get으로 가지고 와야함.
      - detail 페이지로 이동할 땐, 리스트로 화면에 객체들을 출력하고 있고 사용자는 해당 객체를 통해서 이동할 수 있을 때
      - 에러가 발생한다는 것은?
        - 우리가 작성하는 비지니스 로직에서 조건에 맞는 객체가 만드시 하나만 존재해야한다.

- filter는 QuerySet Objet를 반환한다
  - 만약 해당하는 객체가 없으면?
    - 에러가 발생하지 않는다.
    - 빈 QuerySet을 반환한다. (비어있더라고 결과로 생각할 수 있다.)
  - filter로 얻을 수 있는 장점
    - 유연한 처리가 가능
    - 빈 값이라는 데이터도 결과로 치환이 가능 (빈 값에 대한 처리 코드 작성 가능)
- 개발자는 어딴 ORM을 사용할지 결정하는 기준을 무엇으로 잡을까?
  - 구성하려는 애플리케이션의 요구사항, 데이터, 데이터베이스의 설계등을 고려해서 결정해야 한다. 이러한 접근 방식을 통해 예상치 못한 오류나 버그를 미연에 방지한다.

# 단일 객체 조회문
- first()
```sql
SELECT *
FROM books
-- ORDER BY id ASC
LIMIT 1;
```

```python
book = Book.objects.all().first()
print(type(book))
# <class 'books.models.Book'>
```

- last()
```sql
SELECT *
FROM books
ORDER BY id DESC
LIMIT 1;
```

```python
book = Book.objects.all().last()
print(type(book))
# <class 'books.models.Book'>
```

- earliest(field_name) : 필드명을 기준으로 가장 오래된 레코드를 가지고 온다.
```sql
SELECT *
FROM books
ORDER BY created_at ASC
LIMIT 1;
```

```python
book = Book.objects.earliest('created_at')
```

- latest(field_name) : 필드명을 기준으로 가장 최신의 레코드를 가지고 온다.
```sql
SELECT *
FROM books
ORDER BY created_at DESC
LIMIT 1;
```

```python
book = Book.objects.latest('created_at')
```

# 대소관계
__gte, __lte, __gt
> 필드명 뒤에 대소관계를 붙여 사용한다.  
> ex) title__gte, created_at__lte  

- 페이지 수가 100장 이상인 책 조회
```sql
SELECT *
FROM books
WHERE page_count > 100;
```

```python
Book.objects.filter(page_count__gt=100)
```

# 정렬 조회
- ORDER BY
  - 기본값이 아닌 내림차순 정렬하기
   ```sql
   SELECT *
   FROM books
   ORDER BY title DESC;
   ```
   ```python
   Book.objects.all().order_by('-title')
   books = Book.objects.all().order_by('created_at','-title')
   ```
   - Reverse()
     - 이미 조회한 QuerySet 결과를 뒤집는다
     - Book.objects.all().order_by('-title').reverse()

- values()
  - 우리가 지정한 필드만 조회
    - 책의 제목 + 페이지 수만 조회하고 싶다.
      - 단, 조회 결과가 Book 객체가 아닌 dict이라는 점 주의
      ```python
      books = Book.objects.values('title', 'page_count')
      ```

- values_list()
  - values()와 유사하지만 dict대신에 튜플 리스트 형태를 반환
    - 데이터를 리스트에 담으면 데이터 접근이 더 쉽고 유연함
    - flat이라는 옵션이 존재
  ```python
  titles = Book.objects.values_list('title')
  for title in titles:
    print(title)
  ```
  * with flat 옵션
  ```python
  titles = Book.objects.values_list('title', flat=True)
  for title in titles:
    print(title)
  ```

# Agrregation functions (집계 함수)
- Count : 레코드 세기
- Sum : 해당 숫자 필드의 합계
- Avg : 해당 숫자 필드의 평균
- Max : 해당 숫자 필드의 최댓값
- Min : 해당 숫자 필드의 최솟값

```python
from django.db.models import Sum, Count, Avg
total_books = Book.objects.aggregate(total=Count('id'))
print(total_books)
```

# Annotate
```sql
SELECT category, avg(page_count) as avg_pages
FROM books
GROUP BY category
```

- 주석을 다는 것과 같다.
```python
Book.objects.annotate(total_pages = Sum('page_count'))
```

```python
Book.objects.values('category').annotate(total_pages=Sum('page_count')).order_by('category')
```

# Select_related & prefetch_related
- Select_related : 단일 관계에서 FK 1:N
  - 책과 해당 저자 정보를 함께 가져올 때 사용할 수 있다.

- prefetch_related
  - all() 한 뒤에 여기 담긴 pk들을 조회 조건으로 삼아 comment를 모두 조회
    ```sql
    SELECT * FROM books; -- 여기서 나온 쿼리셋에 담긴 pk들을
    SELECT * FROM comments WHERE book.id in -- 쿼리에 담긴 pk들
    ```
  - M to M 또는 역참조 시에 연결된 여러 객체를 가지고 온다.
  - 책에 작성된 리뷰, 책을 조회할 때 리뷰도 다 같이 가지고 온다.